In [ ]:
'''

This program trains a deep learning model using the VGG16 architecture to classify images into two categories: "clear" or "not clear," 
based on the background.
It preprocesses and augments training images, 
uses a pre-trained VGG16 model for feature extraction, and fine-tunes it with 
additional layers to perform binary classification. The model is then trained and saved for future use.
'''
# This is the copy ipynb built on colab 
# Due to lack of data model is overfitting
# We may add more data to make good model

In [3]:
import os

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [7]:
base_dir = "content"  # Base folder
train_dir = os.path.join(base_dir, "train")  # Train directory
test_dir = os.path.join(base_dir, "test")    # Test directory

In [8]:
print(train_dir)

content/train


In [9]:
import os
os.listdir(train_dir)
# os.listdir(test_dir)

['unclear', 'clear']

In [21]:
!rm -rf /content/train/.ipynb_checkpoints
!rm -rf /content/test/.ipynb_checkpoints


In [22]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Train Data Generator (no validation split)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Two classes: clear vs. not_clear
)

# ✅ Data generator for test set (no augmentation, just rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Two classes: clear vs. not_clear
)


Found 320 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [25]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [26]:
for layer in base_model.layers:
    layer.trainable = False

In [27]:
x = Flatten()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)  # Prevent overfitting
x = Dense(1, activation="sigmoid")(x)  # Binary output (0 or 1)

In [28]:
model = Model(inputs=base_model.input, outputs=x)


In [30]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [31]:
EPOCHS = 10
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator  # Use test data as the validation set
)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 926ms/step - accuracy: 0.6688 - loss: 0.6621 - val_accuracy: 0.6250 - val_loss: 0.7090
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.8894 - loss: 0.2889 - val_accuracy: 0.7500 - val_loss: 0.5850
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9162 - loss: 0.2708 - val_accuracy: 0.6250 - val_loss: 0.7917
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9011 - loss: 0.2040 - val_accuracy: 0.5833 - val_loss: 0.8936
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9443 - loss: 0.1953 - val_accuracy: 0.7500 - val_loss: 0.5058
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9197 - loss: 0.1672 - val_accuracy: 0.8333 - val_loss: 0.4547
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8989 - loss: 0.1864 - val_accuracy: 0.5833 - val_loss: 0.7952
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9511 - loss: 0.1637 - val_accuracy: 0.7500 - val_lo

In [32]:
model.save("/content/vgg_background_classifier.h5")